# 【使い方】

ランタイム>すべてのセルを実行（**Ctrl+F9**）によりすべてのセルを実行し、セル[2]の最後に生成された**URL（Running on public URL）をクリック**して開いてください。（GUIが新しいタブで開かれる）

※このcolabの画面（タブ）は閉じないでください。




# [How to use]

Run all cells by selecting Runtime > Run all cells (**Ctrl+F9**), and **click the URL generated at the end of Cell [2] (Running on public URL)** to open it. (The GUI will open in a new tab)

Note: Please do not close this Colab screen (tab).

In [ ]:
!pip install gradio pdf2image
!apt-get install poppler-utils

In [ ]:
import gradio as gr
from PIL import Image
import numpy as np
import os
import zipfile
from pdf2image import convert_from_path

# カラーマスク用の色とカテゴリID
color_to_category_id = {
    (255, 0, 0, 255): 1,     # 赤
    (0, 0, 255, 255): 2,     # 青
    (0, 255, 0, 255): 3,     # 緑
    (255, 255, 0, 255): 4,   # 黄
    (128, 0, 128, 255): 5,   # 紫
    (255, 165, 0, 255): 6,   # オレンジ
    (0, 255, 255, 255): 7,   # 水色
    (173, 255, 47, 255): 8,  # 黄緑
    (128, 128, 128, 255): 9, # グレー
    (0, 128, 128, 255): 10,  # 青緑
    (255, 192, 203, 255): 11,# ピンク
    (255, 20, 147, 255): 12, # ローズ
    (0, 128, 0, 255): 13,    # オリーブ
    (128, 0, 0, 255): 14,    # マルーン
    (0, 255, 230, 255): 15,  # シアン
    (255, 215, 0, 255): 16,  # ゴールド
    (255, 69, 0, 255): 17,   # インディアンレッド
    (0, 0, 128, 255): 18,    # ネイビーブルー
    (220, 20, 60, 255): 19,  # クリムゾンレッド
    (128, 128, 0, 255): 20   # オリーブグリーン
}

# PDFをPNGに変換する関数（修正版）
def convert_pdf_to_png(pdf_file):
    output_folder = '/content/images'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # PDFファイルのページ数を取得
    from pdf2image.pdf2image import pdfinfo_from_path
    pdf_info = pdfinfo_from_path(pdf_file.name)
    total_pages = pdf_info['Pages']

    image_paths = []

    # 各ページを個別に処理してPNGに変換
    for page_num in range(1, total_pages + 1):
        images = convert_from_path(pdf_file.name, first_page=page_num, last_page=page_num)
        image = images[0]  # 1ページのみ変換されるので最初の画像を取得
        image_path = os.path.join(output_folder, f'image{page_num:04}.png')
        image.save(image_path, 'PNG')
        image_paths.append(image_path)

    return image_paths

# 画像を参照画像に合わせてリサイズする関数
def resize_images(image_paths, reference_image):
    ref_img = Image.open(reference_image.name)
    width, height = ref_img.size

    for image_path in image_paths:
        with Image.open(image_path) as img:
            resized_img = img.resize((width, height))
            resized_img.save(image_path)

    return f"リサイズが完了しました: {len(image_paths)} 枚"

# カラー領域を抽出しバイナリ画像にする関数
def extract_and_save_colors(image_paths):
    zip_folder_path = '/content/extracted_colors'
    os.makedirs(zip_folder_path, exist_ok=True)

    for image_index, image_path in enumerate(image_paths):
        image = Image.open(image_path).convert('RGBA')
        data = np.array(image)

        for color, category_id in color_to_category_id.items():
            color_areas = (data[:, :, 0] == color[0]) & (data[:, :, 1] == color[1]) & (data[:, :, 2] == color[2])
            binary_image = np.zeros((data.shape[0], data.shape[1]), dtype=np.uint8)
            binary_image[color_areas] = 255

            label_dir = os.path.join(zip_folder_path, f'label{category_id:02}')
            os.makedirs(label_dir, exist_ok=True)
            file_name = f'image_{image_index + 1:03d}.png'
            Image.fromarray(binary_image).save(os.path.join(label_dir, file_name))

    zip_path = '/content/extracted_colors.zip'
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for root, dirs, files in os.walk(zip_folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, zip_folder_path))

    return zip_path

# Gradioインターフェース
def process_files(pdf_file, reference_image):
    image_paths = convert_pdf_to_png(pdf_file)
    resize_result = resize_images(image_paths, reference_image)
    zip_path = extract_and_save_colors(image_paths)
    return zip_path

# Gradio GUIの作成
with gr.Blocks() as demo:
    pdf_input = gr.File(label="Upload PDF of edited masks")
    ref_image_input = gr.File(label="Upload an image to reference size")
    submit_button = gr.Button("Split Masks")
    output = gr.File(label="Download Splitted masks (ZIP)")

    submit_button.click(fn=process_files,
                        inputs=[pdf_input, ref_image_input],
                        outputs=output)

demo.launch()

Copyright (c) 2024 Satoru Muro. All rights reserved.
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.